In [ ]:
#This code is adapted from Primer of Deep Learning in Genomics
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# The LabelEncoder encodes a sequence of nucleotids bases or amino acids (letters) as a sequence of integers.
integer_encoder = LabelEncoder()
# The OneHotEncoder converts an array of integers to a sparse matrix where
# each row corresponds to one possible value of each feature.
one_hot_encoder = OneHotEncoder(categories='auto')
input_features = []

sequences = ["AAAMGLPVSWAPPALWVLGCCALLLSLWALCTACRRPEDAVAPRKRARRQRARLQGSATAAEASLLRRTHLCSLSKSDTRLHELHRGPRSSRALRPASMDLLRPHWLEVSRDITGPQAAPSAFPHQELPRALPAAAATAGCAGLEATYSNVGLAALPGVSLAASPVVAEYARVQKRKGTHRSPQEPQQGKTEVTPAAQVDVLYSRVCKPKRRDPGPTTDPLDPKGQGAILALAGDLAYQTLPLRALDVDSGPLENVYESIRELGDPAGRSSTCGAGTPPASSCPSLGRGWRPLPASLP"]
for sequence in sequences:
  integer_encoded = integer_encoder.fit_transform(list(sequence))
  integer_encoded = np.array(integer_encoded).reshape(-1, 1)
  one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
  input_features.append(one_hot_encoded.toarray())

print(str(input_features[0]))
print(input_features[0].shape)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(298, 20)


In [ ]:
#Upload the deeploc_data.fasta file

from google.colab import files
uploaded = files.upload()


Saving deeploc_data.fasta to deeploc_data.fasta


In [ ]:
#This code is adapted from RBP_binding_site_prediction in order to read in fasta or faa files.
#This reads in the entire file and saves the labels in the sequence headers as classes.

def read_seq_graphprot(seq_file):
    seq_list = []
    labels = []
    names = []
    seq = ''
    for line2 in seq_file:
            line = line2.decode().strip()
            if line[0] == '>':
                name = line[1:]
                names.append(name)
                label = name.split(" ")[1][:-2]
                labels.append(label)
            else:
                seq = line[:-1].upper()
                #seq = seq.replace('T', 'U')
                seq_list.append(seq)

    return seq_list, labels, names


def read_data_file(file = None, train = True):
    data = dict()
    seqs, labels, names = read_seq_graphprot(file)

    data["seq"] = seqs
    data["class_labels"] = np.array(labels)
    data["names"] = np.array(names)


    return data



In [ ]:
#Preprocess the data
import numpy as np
import io
data = read_data_file(io.BytesIO(uploaded['deeploc_data.fasta']))

In [ ]:
#Check what is in your data
data["seq"][0:7]


['MGLPVSWAPPALWVLGCCALLLSLWALCTACRRPEDAVAPRKRARRQRARLQGSATAAEASLLRRTHLCSLSKSDTRLHELHRGPRSSRALRPASMDLLRPHWLEVSRDITGPQAAPSAFPHQELPRALPAAAATAGCAGLEATYSNVGLAALPGVSLAASPVVAEYARVQKRKGTHRSPQEPQQGKTEVTPAAQVDVLYSRVCKPKRRDPGPTTDPLDPKGQGAILALAGDLAYQTLPLRALDVDSGPLENVYESIRELGDPAGRSSTCGAGTPPASSCPSLGRGWRPLPASL',
 'MEVLEEPAPGPGGADAAERRGLRRLLLSGFQEELRALLVLAGPAFLAQLMMFLISFISSVFCGHLGKLELDAVTLAIAVINVTGISVGHGLSSACDTLISQTYGSQNLKHVGVILQRGTLILLLCCFPCWALFINTEQILLLFRQDPDVSRLTQTYVMVFIPALPAAFLYTLQVKYLLNQGIVLPQVITGIAANLVNALANYLFLHQLHLGVMGSALANTISQFALAIFLFLYILWRKLHHATWGGWSWECLQDWASFLQLAIPSMLMLCIEWWAYEVGSFLSGILGMVELGAQSITYELAIIVYMIPAGFSVAANVRVGNALGAGNIDQAKKSSAISLIVTELFAVTFCVLLLGCKDLVGYIFTTDWDIVALVAQVVPIYAVSHLFEALACTCGGVLRGTGNQKVGAIVNAIGYYVIGLPIGISLMFVAKLGVIGLWSGIIICSVCQTSCFLVFIARLNWKLACQQAQVHANLKVNVALNSAVSQEPAHPVGPESHGEIMMTDLEKKDEIQLDQQMNQQQALPVHPKDSNKLSGKQLALRRGLLFLGVVLVLVGGILVRVYIRT',
 'MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAMRRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKE

In [ ]:
data["class_labels"]


array(['Cell.membrane', 'Cell.membrane', 'Cell.membrane', ...,
       'Extracellular', 'Extracellular', 'Extracellular'], dtype='<U21')

In [ ]:
set(data["class_labels"])

{'Cell.membrane',
 'Cytoplasm',
 'Cytoplasm-Nucleus',
 'Endoplasmic.reticulum',
 'Extracellular',
 'Golgi.apparatus',
 'Lysosome/Vacuole',
 'Mitochondrion',
 'Nucleus',
 'Peroxisome',
 'Plastid'}

In [ ]:
print(len(data['seq']))


14004


In [ ]:
print(len(data["class_labels"]))

14004


In [ ]:
data["names"]


array(['Q9H400 Cell.membrane-M test', 'Q5I0E9 Cell.membrane-M',
       'P63033 Cell.membrane-M', ..., 'P80156 Extracellular-S',
       'Q8NIH1 Extracellular-S', 'D4APA9 Extracellular-S'], dtype='<U37')

**The below is KMeans**

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

# Function to convert protein sequences to k-mer frequency vectors
def sequences_to_kmer_frequencies(sequences, k=3):
    count_vectorizer = CountVectorizer(analyzer='char', ngram_range=(k, k))
    kmer_frequencies = count_vectorizer.fit_transform(sequences)
    return kmer_frequencies.toarray()

# Sequences to k-mer frequency vectors
kmer_frequencies = sequences_to_kmer_frequencies(data["seq"], k=3)

# Standardize the k-mer frequencies
scaler = StandardScaler()
kmer_frequencies_standardized = scaler.fit_transform(kmer_frequencies)


In [ ]:
from sklearn.cluster import MiniBatchKMeans


In [ ]:

# Perform k-means clustering with 11 clusters
n_clusters = 11
minibatch_kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=0, max_iter=100)
minibatch_kmeans.fit(kmer_frequencies_standardized)


# Get cluster assignments for each sequence
cluster_assignments = minibatch_kmeans.labels_





/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
from collections import Counter
from sklearn.metrics import accuracy_score


# Assign the majority class to each cluster
cluster_majority_classes = []
for i in range(n_clusters):
    cluster_member_labels = [data["class_labels"][j] for j in range(len(cluster_assignments)) if cluster_assignments[j] == i]
    majority_class = Counter(cluster_member_labels).most_common(1)[0][0]
    cluster_majority_classes.append(majority_class)

# Predict class labels based on majority class of each cluster
predicted_labels = [cluster_majority_classes[cluster_assignments[i]] for i in range(len(cluster_assignments))]

# Calculate the number of misclassified sequences
misclassified = sum(predicted_labels[i] != data["class_labels"][i] for i in range(len(predicted_labels)))
accuracy = accuracy_score(data["class_labels"], predicted_labels)

print(f"Number of misclassified sequences: {misclassified}")
print(f"Accuracy: {accuracy:.4f}")


Number of misclassified sequences: 9947
Accuracy: 0.2897


In [ ]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(data["class_labels"], predicted_labels, average='macro')
recall = recall_score(data["class_labels"], predicted_labels, average='macro')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Precision: 0.3785
Recall: 0.0916


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**The below is LSTM**

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Read the data
sequences = data["seq"]
labels = data["class_labels"]




In [ ]:
#Using the HW code

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# The LabelEncoder encodes a sequence of nucleotids bases or amino acids (letters) as a sequence of integers.
integer_encoder = LabelEncoder()
# The OneHotEncoder converts an array of integers to a sparse matrix where
# each row corresponds to one possible value of each feature.
one_hot_encoder = OneHotEncoder(categories='auto')
# One-hot encode the sequences
input_features = []
for sequence in sequences:
    integer_encoded = integer_encoder.fit_transform(list(sequence))
    integer_encoded = np.array(integer_encoded).reshape(-1, 1)
    one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
    input_features.append(one_hot_encoded.toarray())

print(str(input_features[0]))
print(input_features[0].shape)





[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(294, 20)


In [ ]:
for i, seq in enumerate(input_features):
    if seq.shape[1] != 20:
        print(f"Sequence {i} has shape {seq.shape}")


Sequence 2 has shape (265, 19)
Sequence 66 has shape (146, 19)
Sequence 77 has shape (319, 19)
Sequence 96 has shape (85, 19)
Sequence 97 has shape (265, 19)
Sequence 181 has shape (99, 18)
Sequence 230 has shape (253, 19)
Sequence 234 has shape (139, 19)
Sequence 365 has shape (227, 19)
Sequence 371 has shape (122, 19)
Sequence 373 has shape (127, 19)
Sequence 396 has shape (474, 19)
Sequence 400 has shape (419, 19)
Sequence 403 has shape (123, 19)
Sequence 406 has shape (192, 19)
Sequence 421 has shape (146, 18)
Sequence 428 has shape (1361, 21)
Sequence 429 has shape (139, 19)
Sequence 437 has shape (74, 13)
Sequence 473 has shape (518, 19)
Sequence 562 has shape (70, 18)
Sequence 603 has shape (138, 18)
Sequence 606 has shape (89, 19)
Sequence 650 has shape (531, 19)
Sequence 671 has shape (112, 16)
Sequence 729 has shape (197, 18)
Sequence 753 has shape (365, 19)
Sequence 786 has shape (424, 19)
Sequence 826 has shape (198, 19)
Sequence 854 has shape (92, 18)
Sequence 934 has shap

In [ ]:
data["class_labels"]

array(['Cell.membrane', 'Cell.membrane', 'Cell.membrane', ...,
       'Extracellular', 'Extracellular', 'Extracellular'], dtype='<U21')

In [ ]:
valid_input_features = [seq for seq in input_features if seq.shape[1] == 20]

original_labels = data["class_labels"]
filtered_labels = []

for i, seq in enumerate(input_features):
    if seq.shape[1] == 20:
        filtered_labels.append(original_labels[i])

assert len(filtered_labels) == len(valid_input_features), "No of filtered labels should match the number of valid input features"
data["class_labels"] = np.array(filtered_labels)


In [ ]:
# Pad the one-hot encoded sequences to the same length
max_length = 300
padded_sequences = pad_sequences(valid_input_features, maxlen=max_length, padding='post', truncating='post', dtype='float32')
print(padded_sequences[:3])


[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score


# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(filtered_labels)
num_classes = len(np.unique(encoded_labels))


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_labels, test_size=0.2, random_state=42)

In [ ]:
# Define the model
model = Sequential()
model.add(Bidirectional(LSTM(64, input_shape=(max_length, 20))))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

# Evaluate the model

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test set accuracy: {accuracy}")
print(f"Test set Precision: {precision}")
print(f"Test set Recall: {recall}")


Epoch 1/5
293/293 [==============================] - 19s 29ms/step - loss: 1.8392 - accuracy: 0.3628 - val_loss: 1.5911 - val_accuracy: 0.4577
Epoch 2/5
293/293 [==============================] - 7s 23ms/step - loss: 1.5120 - accuracy: 0.4794 - val_loss: 1.4057 - val_accuracy: 0.5111
Epoch 3/5
293/293 [==============================] - 7s 22ms/step - loss: 1.4736 - accuracy: 0.4797 - val_loss: 1.7619 - val_accuracy: 0.4204
Epoch 4/5
293/293 [==============================] - 7s 23ms/step - loss: 1.3921 - accuracy: 0.5043 - val_loss: 1.3189 - val_accuracy: 0.5423
Epoch 5/5
74/74 [==============================] - 1s 8ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test set accuracy: 0.473053902387619
Test set Precision: 0.3892202155756144
Test set Recall: 0.47305389221556887


**The below is CNN**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

# CNN Model
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(max_length, 20)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test set accuracy: {accuracy}")
print(f"Test set Precision: {precision}")
print(f"Test set Recall: {recall}")


Epoch 1/5
293/293 [==============================] - 8s 5ms/step - loss: 1.8178 - accuracy: 0.3726 - val_loss: 1.6138 - val_accuracy: 0.4559
Epoch 2/5
293/293 [==============================] - 1s 5ms/step - loss: 1.3684 - accuracy: 0.5414 - val_loss: 1.3979 - val_accuracy: 0.5192
Epoch 3/5
293/293 [==============================] - 1s 4ms/step - loss: 1.1441 - accuracy: 0.6139 - val_loss: 1.3110 - val_accuracy: 0.5342
Epoch 4/5
293/293 [==============================] - 1s 4ms/step - loss: 0.9471 - accuracy: 0.6837 - val_loss: 1.3243 - val_accuracy: 0.5346
Epoch 5/5
74/74 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test set accuracy: 0.5252352356910706
Test set Precision: 0.5391953396923352
Test set Recall: 0.525235243798118


**Commentary: The best result I obtained was from CNN. Compared to LSTM, there were more layers, which may have contributed to better accuracy and precision and recall. the KMeans didn't perform as well as CNN and LSTM because it is a simple clustering algorithm based on the kmers and it doesn't consider sequential information. Also, LSTM and CNN capture non-linear relationship, but Kmeans does not. Lastly, kmeans is an unsupervised ML technique, while LSTM and CNN are supervised. Hence, it is better suited for the classification task.**